## Load Data

Load's all years of the JSON boat data.
Indexes data into boat_index by boat's sail number and type and then stores data for every year.


In [21]:
from __future__ import print_function

import sys
import json
import codecs
import ast
from prettytable import PrettyTable
import pandas as pd

import demjson3
#from barely_json import parse

import re

#from parser.parser import parse_json, parse_json_glob
#from parser.util import log

# This is really ugly code that uses a lot of fragile regex's to convert the 
# JSON files (which aren't legal JSON) into something that Python will parse
def load_data(filename):
    s = codecs.open(filename, "r", "utf-8-sig").read()

    # replace ROCK''N''ROLL with ROCKNROLL
    s = re.sub(': \"([^"]*)\'\'([^\'\"]+)\'\'([^"]*)\"', ": \'\\1\\2\\3\'", s)
    # replace "foo's'blah" with "foosblah"
    s = re.sub(': \"([^"]*)\'([^\'\"]+)\'([^"]*)\"', ": \'\\1\\2\\3\'", s)
    # replace "foo's blah" with "foos blah"
    s = re.sub(': \"([^"]*)\'([^"]*)\"', ": \'\\1\\2\'", s)
    # replace 'foo "bar"' with 'foo bar'
    s = re.sub(': \'([^\'"]*)"([^"]*)"([^\']*)\'', ": \'\\1\\2\\3\'", s)

    s = re.sub(': None,', ': "None",', s)

    # replace ' with "
    s = re.sub('(?<!\\\\)\'', '\"', s)
    # replace , NN: with , "NN":
    s = re.sub(',\s+([0-9]+):', ", \"\\1\":", s)
    s = s.replace('(', '[')
    s = s.replace(')', ']')
    s = re.sub('\\\\', '', s)
    boats = json.loads(s)
    return boats

# This loads all of the JSON files into the "boats" dict
# The structure this is:
# boats[year] -> [ boat1, boat2, boat3, ... boatN ]
# The boat data is a direct translation from the JSON files
boats = {}
for i in range(2019,2024):
    filename = "ALL" + str(i) + ".json"
    print("loading from " + filename)
    boats[i] = load_data(filename)

# This converts the data into a boats_index dicts.
# The structure is:
# boats_index[<sailnu><boattype>]['years'][year] -> { <year-based data dict> }
# boats_index[<sailnu><boattype>]['keys'] -> { <global boat info data dict> }
# fields in the data dict are in the list below
# The goal of boat_index is to make it easy to see all of the data that we 
# care about indexed by the boat instead of by year.
boat_index = {}
for year in range(2019, 2024):
    for boat in boats[year]:
        key = (boat['sailnumber'], boat['boat']['type'])
        boat_data = {
            "name": boat['name'],
            "sail_number": boat['sailnumber'],
            "gph": boat['rating']['gph'], 
            "type": boat['boat']['type'], 
            "crew_weight": boat['boat']['sizes']['crew'],
            "spin_sqm": boat['boat']['sizes']['spinnaker'], 
            "asym_sqm": boat['boat']['sizes']['spinnaker_asym'],
            "8kt_run_angle": boat['vpp']['run_angle'][1],
            "8kt_run_vmg": boat['vpp']['run_vmg'][1],
            "10kt_run_angle": boat['vpp']['run_angle'][2],
            "10kt_run_vmg": boat['vpp']['run_vmg'][2],
            "12kt_run_angle": boat['vpp']['run_angle'][3],
            "12kt_run_vmg": boat['vpp']['run_vmg'][3],
            "14kt_run_angle": boat['vpp']['run_angle'][4],
            "14kt_run_vmg": boat['vpp']['run_vmg'][4],    
            "vpp": boat['vpp']
        }
        if not (key in boat_index):
            boat_index[key] = {} 
            boat_index[key]['years'] = {}
            boat_index[key]['keys'] = {}
        boat_index[key]['years'][year] = boat_data


loading from ALL2019.json
loading from ALL2020.json
loading from ALL2021.json
loading from ALL2022.json
loading from ALL2023.json


## Find boats that have changed spinnaker types

Look for boats that have had a symmetrical spinnaker only in one year and an asymmetrical spinnaker only in another year. Filter out any where the crew weight has changed. Store the output into interesting_boats.

In [22]:
def find_interesting_boats(boat_index):
    interesting_boats = []
    for boat_key in boat_index:
        boat = boat_index[boat_key]
        spin_only = False
        asym_only = False
        crew_min = 100000
        crew_max = 0
        for year in boat['years']:
            if boat['years'][year]['spin_sqm'] == 0 and boat['years'][year]['asym_sqm'] > 0:
                asym_only = True
                boat['keys']['asym'] = year
            if boat['years'][year]['spin_sqm'] > 0 and boat['years'][year]['asym_sqm'] == 0:
                spin_only = True
                boat['keys']['spin'] = year
            if boat['years'][year]['crew_weight'] < crew_min:
                crew_min = boat['years'][year]['crew_weight']
            if boat['years'][year]['crew_weight'] > crew_max:
                crew_max = boat['years'][year]['crew_weight']
            
        #print(boat)
        #print(spin_min, spin_max, asym_min, asym_max)
        #return
        #if (asym_min == 0 and asym_max > 0) or (spin_min == 0 and spin_max > 0):
        #    interesting_boats.append(boat)
        if spin_only and asym_only and (crew_min == crew_max):
            interesting_boats.append(boat)
    return interesting_boats
            
interesting_boats = find_interesting_boats(boat_index)
print(len(interesting_boats))



38


# Produce Table

Produce's a table that can be loaded in Excel for all of the candidate boats

In [23]:
def export_summary(interesting_boats):
    table = {
        "sail_num": [],
        "boat_type": [],
        "orc_years": [],
        "spin_sqm": [],
        "asym_sqm": [],
        "spin_vmg_8kts": [],
        "asym_vmg_8kts": [],
        "asym_advantage_8kts": [],
        "spin_vmg_10kts": [],
        "asym_vmg_10kts": [],
        "asym_advantage_10kts": [],
        "spin_vmg_12kts": [],
        "asym_vmg_12kts": [],
        "asym_advantage_12kts": [],
        "spin_vmg_14kts": [],
        "asym_vmg_14kts": [],
        "asym_advantage_14kts": []
    }
    for boat in interesting_boats:
        spin_vmg_8 = 0
        asym_vmg_8 = 0
        spin_vmg_10 = 0
        asym_vmg_10 = 0
        spin_vmg_12 = 0
        asym_vmg_12 = 0
        spin_vmg_14 = 0
        asym_vmg_14 = 0
        boat_sail_number = ""
        boat_name = ""
        boat_type = ""
        spin_sqm = 0
        asym_sqm = 0
        years = []
        for year in boat['years']:
            boat_year = boat['years'][year]
            boat_sail_number = boat_year['sail_number']
            boat_type = boat_year['type']
            boat_name = boat_year['name']
            sail_type = ""
            if not (boat_year['spin_sqm'] > 0 and boat_year['asym_sqm'] > 0):
                if boat_year['spin_sqm'] > 0:
                    spin_vmg_8 = boat_year['8kt_run_vmg']
                    spin_vmg_10 = boat_year['10kt_run_vmg']
                    spin_vmg_12 = boat_year['12kt_run_vmg']
                    spin_vmg_14 = boat_year['14kt_run_vmg']
                    spin_sqm = boat_year['spin_sqm']
                    sail_type = "s"
                if boat_year['asym_sqm'] > 0:
                    asym_vmg_8 = boat_year['8kt_run_vmg']
                    asym_vmg_10 = boat_year['10kt_run_vmg']
                    asym_vmg_12 = boat_year['12kt_run_vmg']
                    asym_vmg_14 = boat_year['14kt_run_vmg']
                    asym_sqm = boat_year['asym_sqm']
                    sail_type = "a"
                years.append(str(year) + ":" + sail_type)
        
        #if boat_sail_number == "NED/NED4937":
        #    print(boat)
        
        table["sail_num"].append(boat_sail_number)
        table["boat_type"].append(boat_type)
        table["orc_years"].append(years)
        table["spin_sqm"].append(spin_sqm)
        table["asym_sqm"].append(asym_sqm)
        table["spin_vmg_8kts"].append(spin_vmg_8)
        table["asym_vmg_8kts"].append(asym_vmg_8)
        table["asym_advantage_8kts"].append(asym_vmg_8 - spin_vmg_8)
        table["spin_vmg_10kts"].append(spin_vmg_10)
        table["asym_vmg_10kts"].append(asym_vmg_10)
        table["asym_advantage_10kts"].append(asym_vmg_10 - spin_vmg_10)
        table["spin_vmg_12kts"].append(spin_vmg_12)
        table["asym_vmg_12kts"].append(asym_vmg_12)
        table["asym_advantage_12kts"].append(asym_vmg_12 - spin_vmg_12)
        table["spin_vmg_14kts"].append(spin_vmg_14)
        table["asym_vmg_14kts"].append(asym_vmg_14)
        table["asym_advantage_14kts"].append(asym_vmg_14 - spin_vmg_14)
        
        
    df = pd.DataFrame(table)
    df.to_excel(r'export_dataframe.xlsx', index=False)
    # too many columns to print nicely in jupyter, but you can uncomment this for debugging
    #print(df)
    
export_summary(interesting_boats)

In [24]:
# This helper function provides various ways of filtering out a boat or set of boats and is used in the 
# next group of cells

# filter the full boat list down to some that meet some filters
# filter_type_re == regular express object matched against boat->type
# filter_name == substring search in name
# filter_year == exact match of year
# min/max spin/asym == kite sizes
# TODO -- This could be so much more generic...
def find_boats(boats, filter_type_re=None, filter_name=None, filter_year=None, min_asym=0, max_asym=0, min_spin=0, max_spin=0, print_table=True):
    table = {
        "sail_num": [],
        "name": [],
        "boat_type": [],
        "orc_year": [],
        "spin_sqm": [],
        "asym_sqm": [],
    }
    
    boat_list = []
    i = 0

    for year in boats:
        if filter_year != None and filter_year != year:
            continue
        for boat in boats[year]:
            if filter_name != None and filter_name not in boat['name']:
                continue
                
            if filter_type_re != None and filter_type_re.match(boat['boat']['type']) == None:
                continue
                
            spin_sqm = boat['boat']['sizes']['spinnaker']
            asym_sqm = boat['boat']['sizes']['spinnaker_asym']
            if spin_sqm >= min_spin and spin_sqm <= max_spin and asym_sqm >= min_asym and asym_sqm <= max_asym:
                table["sail_num"].append(boat['sailnumber'])
                table["name"].append(boat['name'])
                table["boat_type"].append(boat['boat']['type'])
                table["orc_year"].append(str(year))
                table["spin_sqm"].append(boat['boat']['sizes']['spinnaker'])
                table["asym_sqm"].append(boat['boat']['sizes']['spinnaker_asym'])
                boat_list.append(boat)
    
    if print_table:
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 120)
        pd.set_option('display.max_colwidth', 15)
        df = pd.DataFrame(table)
        print(df.sort_values(by=["sail_num", "orc_year"]))
    
    return boat_list
    


In [25]:
# This set of functions can average a list of boats into a single boat.  Useful for comparing
# traits across a class

def average_in_vector(output_vector, input_vector, vector_count):
    assert(len(output_vector) == len(input_vector))
    for i in range(0, len(output_vector)):
        output_vector[i] += input_vector[i] / vector_count
    
def average_in_dict(output_dict, input_dict, dict_count):
    for key in input_dict.keys():
        if key not in output_dict:
            output_dict[key] = 0.0
        output_dict[key] += input_dict[key] / dict_count
    
def average_boats(boats, output_boat_name, output_boat_type, output_boat_sail_number):
    boat_count = len(boats)
    output_boat = {} 
    output_boat['sailnumber'] = output_boat_sail_number
    output_boat['boat'] = {}
    output_boat['boat']['type'] = output_boat_type
    output_boat['boat']['sizes'] = {}
    output_boat['name'] = output_boat_name
    for i in range(0, len(boats)):
        boat = boats[i]
        # fields in vpp:
        # angles -- copy from first boat
        # speeds -- copy from first boat
        # NN -- targets for each wind speed at TWA NN [average each element]
        # beat_angle -- upwind angle for wind speeds [average each element]
        # beat_vmg -- upwind speed for wind speeds [average each element]
        # run_angle -- downwind angle for wind speeds [average each element]
        # run_vmg -- downwind speed for wind speeds [average each element]
        #print(boat)
        vpp = boat['vpp']
        if 'vpp' not in output_boat:
            output_boat['vpp'] = {}
            output_boat['vpp']['angles'] = boat['vpp']['angles']
            output_boat['vpp']['speeds'] = boat['vpp']['speeds']
        for entry in vpp.keys():
            if (entry != "angles") and (entry != "speeds"):
                if not entry in output_boat['vpp']:
                    output_boat['vpp'][entry] = [0] * len(vpp[entry])
                average_in_vector(output_boat['vpp'][entry], boat['vpp'][entry], boat_count)
        average_in_dict(output_boat['boat']['sizes'], boat['boat']['sizes'], boat_count)
    return output_boat
    


In [26]:
#
# helper functions for plotting polar charts for a boat or a comparison across two boats
#

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import copy
import plotly.io as pio
import math
pio.renderers.default='notebook_connected'

#
# the VPP uses boat speeds at all points of sail except for beat_vmg and run_vmg where
# it uses VMG.  This helper function converts VMG back to boat speed
#
def inverse_vmg(vmg, angle_degrees):
    angle = math.radians(angle_degrees)
    return vmg * 1 / math.cos(angle)
 
#
# This produces a polar plot for 1 or 2 VPPs from boat certificates
# 
# I use this to overlay a polar plot for sym and asym versions of the same boat to 
# be able to compare results
#
def plot_polar(name, tags, vpps):
    fig = go.Figure()
    colors = [
        ['darkviolet', 'darkgreen', 'darkorange', 'darkcyan', 'darkred', 'darkblue', 'black'],
        ['violet', 'lightgreen', 'yellow', 'cyan', 'red', 'blue', 'grey']]

    # we don't have enough colors defined to handle vpps for more than 2 boats
    assert(len(vpps) <= len(colors))
    for vpp_index in range(0, len(vpps)):
        vpp = vpps[vpp_index]
        for speed_index in range(0, len(vpp['speeds'])):
            beat_angle = vpp['beat_angle'][speed_index]
            run_angle = vpp['run_angle'][speed_index]
            angles = [ beat_angle ]
            angles.extend(vpp['angles'])
            insert_index = 0
            # insert the run_vmg at the proper point in the angles[] list.  
            # keep track of where we inserted to do the same for speeds
            # insert_index == 0 means append
            for angle_index in range(0, len(angles)):
                if run_angle < angles[angle_index]:
                    insert_index = angle_index
                    angles.insert(insert_index, run_angle)
            if (insert_index == 0):
                angles.append(run_angle)
            #print(angles)

            speeds = [ inverse_vmg(vpp['beat_vmg'][speed_index], beat_angle) ]
            for angle in vpp['angles']:
                speeds.append(vpp[str(angle)][speed_index])
            if insert_index == 0:
                speeds.append(inverse_vmg(vpp['run_vmg'][speed_index], 180 - run_angle))
            else:
                speeds.insert(insert_index, inverse_vmg(vpp['run_vmg'][speed_index], 180 - run_angle))
            #print(speeds)
            
            marker_sizes = [ 10 ]
            marker_sizes.extend([6] * (len(angles) - 2))
            if insert_index == 0:
                marker_sizes.append(10)
            else:
                marker_sizes.insert(insert_index, 10)
            
            marker_shapes = [ "diamond" ]
            marker_shapes.extend(["circle"] * (len(angles) - 2))
            if insert_index == 0:
                marker_shapes.append("diamond")
            else:
                marker_shapes.insert(insert_index, "diamond")

            fig.add_trace(
                go.Scatterpolar(
                    r = speeds,
                    theta = angles,
                    mode = 'lines+markers+text',
                    marker = dict(size = marker_sizes, symbol = marker_shapes),
                    line = dict(shape="spline"),
                    name = str(vpp['speeds'][speed_index]) + ":" + tags[vpp_index],
                    line_color = colors[vpp_index][speed_index]))
    fig.update_layout(
        title=name,
        showlegend=True,
        height=640,
        polar = dict(
            domain = dict(x = [1, 0], y = [0, 1]),
            sector = [-90, 90],
            angularaxis = dict(thetaunit = "degrees", dtick = 10, rotation=90, direction="clockwise")
        ))
    fig.show()
        
    return

def compare_two_boats(plot_name, boat1, boat2, boat1tag=None, boat2tag=None):
    if boat1tag == None:
        boat1tag = boat1['name']
    if boat2tag == None:
        boat2tag = boat2['name']
    plot_polar(plot_name, [ boat1tag, boat2tag ], [ boat1['vpp'], boat2['vpp'] ])


In [79]:
#
# This cell is comparing all boats in a class by dividing the class into boats that use
# sym vs asym kites.  It averages the VPP data across each of those groups and the 
# prints a comparison plot of the VPPs.
#
# This is probably not very accurate, but it can give an idea of trends across the sail
# sizes and shapes.
#

def dicts_to_table(d1header, d1, d2header, d2):
    table = {
        "key": [ k for k in d1.keys() ],
        d1header: [ d1[k] for k in d1.keys() ],
        d2header: [ d2[k] for k in d2.keys() ]
    }
    return table

def pretty_print_table(t, caption):
    df = pd.DataFrame(t)
    display(df.style \
        .set_caption(caption) \
        .format(precision=2, thousands=",", decimal=".") \
        .format_index(str.upper, axis=1))
        

def compare_average_sym_asym(boat_type_re, plot_title, max_spin, max_asym):
    sym_boats = find_boats(boats, boat_type_re, max_spin=max_spin, print_table=False)
    asym_boats = find_boats(boats, boat_type_re, max_asym=max_asym, print_table=False)
    if len(sym_boats) == 0 or len(asym_boats) == 0:
        print("no boats!", len(sym_boats), len(asym_boats))
        return
    average_asym = average_boats(asym_boats, "asym", "", "XXX/AAA")
    average_sym = average_boats(sym_boats, "sym", "", "XXX/SSS")
    average_asym['boat']['sizes']['boat count'] = len(asym_boats)
    average_sym['boat']['sizes']['boat count'] = len(sym_boats)
    sizes_table = dicts_to_table("asym", average_asym['boat']['sizes'], "sym", average_sym['boat']['sizes'])
    pretty_print_table(sizes_table, plot_title)
    compare_two_boats(plot_title, average_asym, average_sym)
    

compare_average_sym_asym(re.compile('first 36.7', re.IGNORECASE), "First 36.7 Sym/Asym comparison", 100, 100)
compare_average_sym_asym(re.compile('J.*109', re.IGNORECASE), "J/109 Sym/Asym comparison", 9999, 9999)
compare_average_sym_asym(re.compile('J.*105', re.IGNORECASE), "J/105 Sym/Asym comparison", 9999, 9999)
compare_average_sym_asym(re.compile('J.*97', re.IGNORECASE), "J/97 Sym/Asym comparison", 9999, 9999)
compare_average_sym_asym(re.compile('Sun.*fast.*3200', re.IGNORECASE), "Sunfast 3200 Sym/Asym comparison", 9999, 9999)


,KEY,ASYM,SYM
0,loa,10.64,10.66
1,beam,3.46,3.46
2,draft,2.23,2.21
3,displacement,"5,595.81","5,627.89"
4,genoa,33.78,38.84
5,main,38.72,39.64
6,spinnaker,0.00,83.86
7,spinnaker_asym,81.04,0.00
8,crew,539.00,633.03
9,wetted_surface,26.42,26.59


,KEY,ASYM,SYM
0,loa,10.76,10.76
1,beam,3.52,3.52
2,draft,2.13,2.13
3,displacement,"5,070.08","4,973.93"
4,genoa,33.56,38.17
5,main,37.69,37.43
6,spinnaker,0.00,89.33
7,spinnaker_asym,110.39,0.00
8,crew,584.42,604.36
9,wetted_surface,26.15,26.00


,KEY,ASYM,SYM
0,loa,10.57,10.50
1,beam,3.35,3.35
2,draft,2.01,2.01
3,displacement,"4,001.76","4,000.00"
4,genoa,27.49,27.05
5,main,33.93,34.96
6,spinnaker,0.00,90.75
7,spinnaker_asym,94.65,0.00
8,crew,529.42,514.00
9,wetted_surface,24.70,24.89


,KEY,ASYM,SYM
0,loa,9.60,9.62
1,beam,3.35,3.35
2,draft,1.91,2.06
3,displacement,"3,777.80","3,732.00"
4,genoa,24.61,22.38
5,main,28.66,28.99
6,spinnaker,0.00,81.31
7,spinnaker_asym,95.04,0.00
8,crew,526.50,530.00
9,wetted_surface,21.11,21.01


,KEY,ASYM,SYM
0,loa,9.79,9.79
1,beam,3.46,3.46
2,draft,1.92,1.91
3,displacement,"3,696.78","3,622.21"
4,genoa,26.67,25.19
5,main,31.41,31.21
6,spinnaker,0.00,82.74
7,spinnaker_asym,85.57,0.00
8,crew,534.22,509.96
9,wetted_surface,22.79,23.06


In [71]:
#
# The 36.7 Enigma is in all 5 years of data with the same configuration.  
#
# I added this to see how the VPP changes year over year
#
enigma = {}
for year in range(2019, 2024):
    enigma[year] = find_boats(boats, re.compile("first 36.7", re.IGNORECASE), filter_name="Enigma", filter_year=year, max_spin=95, print_table=False)[0]

for year in range(2019, 2023):
    compare_two_boats("Enigma " + str(year) + " vs " + str(year+1), enigma[year], enigma[year+1], boat1tag=str(year), boat2tag=str(year+1))


In [80]:
# this prints all boats that changed kite types
#
# The VPP can change year by year, but as shown above the changes are pretty minor for the
# classes of racer/cruisers that I'm primarily interested in.

def plot_interesting_boats(interesting_boats):
    for boat in interesting_boats:
        asym_year = boat['keys']['asym']
        spin_year = boat['keys']['spin']
        tags = ["asym:" + str(asym_year), "spin:" + str(spin_year)]
        vpps = [boat['years'][asym_year]['vpp'], boat['years'][spin_year]['vpp']]
        name = "type:" + boat['years'][asym_year]['type'] + ", sailno:" + boat['years'][asym_year]['sail_number'] + ", spin_sqm:" + str(boat['years'][spin_year]['spin_sqm']) +", asym_sqm:" + str(boat['years'][asym_year]['asym_sqm'])
        plot_polar(name, tags, vpps)

        
plot_interesting_boats(interesting_boats)
